In [1]:
#dataset by Prajna Bhandary
#reference from pyimagesearch

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import imutils
#import argparse
import os

from sklearn.preprocessing import LabelBinarizer
#from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.layers import Flatten, Dense, Input, Dropout, AveragePooling2D

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from keras.callbacks import TensorBoard, ModelCheckpoint

%matplotlib inline

In [11]:
train_generator = ImageDataGenerator(rescale = 1.0/255,
                                  rotation_range = 40,
                                  width_shift_range= 0.2,
                                  height_shift_range = 0.2,                                  
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip=True,
                                  fill_mode='nearest')

train_data = train_generator.flow_from_directory('./train',
                                                   batch_size = 10,
                                                   target_size = (224, 224))


Found 1315 images belonging to 2 classes.


In [12]:
test_generator = ImageDataGenerator(rescale = 1.0/255)
test_data = test_generator.flow_from_directory('./test',
                                               batch_size = 10,
                                               target_size = (224,224))


Found 194 images belonging to 2 classes.


In [13]:
learning_rate = 0.001
epochs = 15
batch_size = 32

In [15]:
#using MobileNetV2

baseModel = MobileNetV2(weights='imagenet', include_top=False,
                       input_tensor=Input(shape=(224, 224, 3)))

baseModel.trainable = False

In [17]:
model = Sequential([
    baseModel,
    
    AveragePooling2D(pool_size=(7,7)),
    Flatten(),
    
    Dense(128, activation = "relu"),
    Dropout(0.5),
    Dense(2, activation="softmax")
])

#opt = Adam(lr=learning_rate, decay=learning_rate / epochs)
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

In [20]:
checkpoint = ModelCheckpoint('model_{epoch:02d}.model', 
                            monitor = 'val_loss',
                            verbose = 0,
                            save_best_only=True,
                            mode='auto')

history = model.fit_generator(train_data,
                             epochs=15,
                             validation_data=test_data,
                             callbacks=[checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
132/132 [==============================] - ETA: 0s - loss: 0.1308 - accuracy: 0.9445WARNING:tensorflow:From c:\mysoftwares\python3\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model_01.model\assets
132/132 [==============================] - 161s 1s/step - loss: 0.1308 - accuracy: 0.9445 - val_loss: 0.0434 - val_accuracy: 0.9794
Epoch 2/15
132/132 [==============================] - 146s 1s/step - loss: 0.0657 - accuracy: 0.9787 - val_loss: 0.0121 - val_accuracy: 0.9948
Epoch 3/15
132/132 [===

In [21]:
model.save("best_model.model", save_format="h5")